In [1]:
import pandas as pd
import unicodedata

# 0 Funcoes

In [2]:
# Como estamos importando de um excel, vamos ter que ajustar o nome das colunas
def normalizar_colunas(df):
    def remover_acentos(texto):
        return ''.join(
            c for c in unicodedata.normalize('NFKD', texto)
            if not unicodedata.combining(c)
        )

    df = df.copy()
    df.columns = [
        remover_acentos(col)
            .lower()
            .replace(' ', '_')
        for col in df.columns
    ]

    return df


In [3]:
# Como vamos juntar alguns dataframes, é melhor que estejam com os mesmos nomes algumas colunas
def renomear_colunas(df, mapa_colunas):
    """
    Parâmetros:
    df (pd.DataFrame): DataFrame original
    mapa_colunas (dict): {'nome_antigo': 'nome_novo'}

    Retorna:
    pd.DataFrame: DataFrame com colunas renomeadas
    """
    df = df.copy()

    # Aplica somente às colunas que existem no DataFrame
    mapa_valido = {
        col_antiga: col_nova
        for col_antiga, col_nova in mapa_colunas.items()
        if col_antiga in df.columns
    }

    df.rename(columns=mapa_valido, inplace=True)
    return df


In [4]:
"""
Como os dataframes tem colunas diferentes, na hora de juntar essa função irá ajudar.
Iremos adicionar colunas nos outros dataframes para que a junção possa ocorrer (as colunas terão dasdos vazios)
"""
def adaptar_dataframe(df, colunas_base, origem, lista_ids):
    df=df.copy()

     # filtra apenas os IDs desejados
    df = df[df['ra'].isin(lista_ids)]

    # adiciona colunas que faltam
    for col in colunas_base:
        if col not in df.columns:
            df[col] = pd.NA

    # mantém apenas as colunas do principal
    df = df[colunas_base]

    # cria coluna de origem
    df['origem_df'] = origem

    return df


# 1 Importando os dados

In [ ]:
arquivo = r"https://github.com/vbomura/tc5/blob/0afbe2d22ee4658c81a7470b2360a2aef3cd032e/Base_Passos_Magicos/BASE%20DE%20DADOS%20PEDE%202024%20-%20DATATHON.xlsx"

# Pegando o dado de cada aba
base_2022 = pd.read_excel(arquivo, sheet_name="PEDE2022")
base_2023 = pd.read_excel(arquivo, sheet_name="PEDE2023")
base_2024 = pd.read_excel(arquivo, sheet_name="PEDE2024")

## Ajustes nas colunas

In [6]:
# Vamos padronizar os nomes das colunas
base_2022 = normalizar_colunas(base_2022)
base_2023 = normalizar_colunas(base_2023)
base_2024 = normalizar_colunas(base_2024)

In [7]:
# Renomeando algumas colunas das bases
mapa = {
    'pedra_23': 'ignorar' #tem uma coluna "repetida no dataframe, vamos ignorar ela pq não tem a info que precisamos"
    ,'pedra_2023': 'pedra_23'
    ,'inde_23': 'ignorar2'
    ,'inde_2023': 'inde_23'
}
base_2023 = renomear_colunas(base_2023, mapa)

mapa = {
    'portug': 'por'
    ,'ingles': 'ing'
    ,'matem': 'mat'
    ,'defas': 'defasagem'
    ,'idade_22': 'idade'
    
}
base_2022 = renomear_colunas(base_2022, mapa)


## Juntando os dataframes

In [8]:
# Vamos juntar os dataframes, mas primeiro vamos evitar juntar casos que já estão 2024
ra_principal = set(base_2024['ra'])

base_2023_filtrado = base_2023[
    (~base_2023['ra'].isin(ra_principal)) #alguns valores estão com problemas,
]

ra_principal.update(base_2023_filtrado['ra'].unique()) #marca os de 2023 para evitar duplicados que tem apenas entre 23 e 22

base_2022_filtrado = base_2022[
    (~base_2022['ra'].isin(ra_principal))
]

# Vamos guardar os id em listas para usar depois
ra_2023_diferentes = base_2023_filtrado['ra'].unique().tolist()
ra_2022_diferentes = base_2022_filtrado['ra'].unique().tolist()

In [9]:
# listamos as colunas do dataframe principal (ano de 2024)
colunas_finais = list(base_2024.columns) + ['ano_dataframe']

# Colocamos as colunas nos dataframes, para que eles fiquem 'parecidos' com 2024, e também uma coluna para saber de qual ano veio a info
base_2024['ano_dataframe'] = 'principal'
base_2023 = adaptar_dataframe(base_2023,colunas_finais, 2023, ra_2023_diferentes)
base_2022 = adaptar_dataframe(base_2023,colunas_finais, 2022, ra_2022_diferentes)

In [ ]:
# Após todos os tratamentos, juntamos tudo
df_unificado = pd.concat(
    [base_2024, base_2023, base_2022],
    ignore_index=True
)

In [12]:
df_unificado.describe()

,ano_ingresso,inde_22,inde_23,cg,cf,ct,no_av,rec_av1,rec_av2,iaa,...,ing,indicado,atingiu_pv,ipv,ian,defasagem,destaque_ieg,destaque_ida,destaque_ipv,origem_df
count,1405.000000,604.000000,931.000000,0.0,0.0,0.0,1401.000000,0.0,0.0,1299.000000,...,580.000000,0.0,0.0,1299.000000,1405.000000,1405.000000,0.0,0.0,0.0,249.0
mean,2022.365836,7.253049,7.342309,NaN,NaN,NaN,2.995004,NaN,NaN,8.184076,...,6.469224,NaN,NaN,7.434301,7.517794,-0.488256,NaN,NaN,NaN,2023.0
std,1.370206,0.910221,0.901757,NaN,NaN,NaN,1.315557,NaN,NaN,2.206085,...,2.807066,NaN,NaN,1.048087,2.535300,0.873419,NaN,NaN,NaN,0.0
min,2016.000000,3.031806,3.745542,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,...,0.000000,NaN,NaN,2.943333,2.500000,-4.000000,NaN,NaN,NaN,2023.0
25%,2021.000000,6.736019,6.724150,NaN,NaN,NaN,2.000000,NaN,NaN,7.917000,...,4.500000,NaN,NaN,6.873750,5.000000,-1.000000,NaN,NaN,NaN,2023.0
50%,2023.000000,7.394350,7.408033,NaN,NaN,NaN,3.000000,NaN,NaN,8.751000,...,7.000000,NaN,NaN,7.500000,10.000000,0.000000,NaN,NaN,NaN,2023.0
75%,2023.000000,7.898334,7.996083,NaN,NaN,NaN,4.000000,NaN,NaN,9.500000,...,9.000000,NaN,NaN,8.166667,10.000000,0.000000,NaN,NaN,NaN,2023.0
max,2024.000000,9.441522,9.371200,NaN,NaN,NaN,6.000000,NaN,NaN,10.002000,...,10.000000,NaN,NaN,10.010000,10.000000,3.000000,NaN,NaN,NaN,2023.0


In [15]:
df_unificado.head()

,ra,fase,inde_2024,pedra_2024,turma,nome_anonimizado,data_de_nasc,idade,genero,ano_ingresso,...,fase_ideal,defasagem,destaque_ieg,destaque_ida,destaque_ipv,escola,ativo/_inativo,ativo/_inativo.1,ano_dataframe,origem_df
0,RA-1275,ALFA,7.611367,Ametista,ALFA A - G0/G1,Aluno-1275,2016-07-28 00:00:00,8,Masculino,2024,...,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando,principal,NaN
1,RA-1276,ALFA,8.002867,Topázio,ALFA A - G0/G1,Aluno-1276,2016-10-16 00:00:00,8,Feminino,2024,...,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando,principal,NaN
2,RA-1277,ALFA,7.9522,Ametista,ALFA A - G0/G1,Aluno-1277,2016-08-16 00:00:00,8,Masculino,2024,...,ALFA (1° e 2° ano),0,NaN,NaN,NaN,EE Dom Pedro Villas Boas de Souza,Cursando,Cursando,principal,NaN
3,RA-868,ALFA,7.156367,Ametista,ALFA A - G0/G1,Aluno-868,2015-11-08 00:00:00,8,Masculino,2023,...,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EE Chácara Florida II,Cursando,Cursando,principal,NaN
4,RA-1278,ALFA,5.4442,Quartzo,ALFA A - G0/G1,Aluno-1278,2015-03-22 00:00:00,9,Masculino,2024,...,Fase 1 (3° e 4° ano),-1,NaN,NaN,NaN,EM Etelvina Delfim Simões,Cursando,Cursando,principal,NaN
